In [ ]:
# ---------- visualizzo piu output a schermo insieme ------------------------------
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"
#-------------------------- scelgo se usare CPU o GPU -----------------------------
import tensorflow as tf
import numpy as np
import os

SEED = 1234
tf.random.set_seed(SEED)
cwd = os.getcwd()

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0' # CPU = -1 , GPU = 0

gpus = tf.config.experimental.list_physical_devices('GPU')
cpus = tf.config.experimental.list_physical_devices('CPU')

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
elif cpus:
    try:
        logical_cpus= tf.config.experimental.list_logical_devices('CPU')
        print(len(cpus), "Physical CPU,", len(logical_cpus), "Logical CPU")
    except RuntimeError as e:
        print(e)
#---------------------------------------------------------------------------

In [ ]:
#---------------- scelta parametri -----------------------------------------
img_h = 192
img_w = 256
N_classes = 4

# scala a cui va ridotta una singola patch dell'immagine
LUT_scale = {"BipbipHaricot": 2,
             "BipbipMais": 4,
             "PeadHaricot": 1,
             "PeadMais": 1,
             "RoseauHaricot": 1,
             "RoseauMais": 1,
             "WeedelecHaricot": 4,
             "WeedelecMais": 4}

# immagini come multipli di patch standard 192x256
TEAM_SIZES = {"Bipbip": [(2048, 1536), 8],  # sacale = 1 + scale = 2
                  "Pead": [(3072, 2304), 12],
                  "Roseau": [(1280, 960), 5],
                  "Weedelec": [(5120, 3840), 20]}

BS = 3
split = 0.8
EP = 10 

In [ ]:
from PIL import Image
# preparo le patches dei diversi team / crop dividendo immagini e maschere del training set escludendo patches completamente nere

prepare_patches = True

if prepare_patches:
    
    teams = ["Pead"] # <-- scegli team
    crops = ["Haricot"] # <-- scegli crop
    
    PSI = 0.95 # <-- scegli quali patche scartare in base a percentuale massima background

    source_dir = os.path.join(cwd,'Development_Dataset', 'Training')
    patch_dir = os.path.join(cwd, 'Patches')
    if not os.path.exists(patch_dir):
        os.makedirs(patch_dir)

    for team in teams:
        for crop in crops:
            scale = LUT_scale[team + crop]
            filenames = os.listdir(os.path.join(source_dir, team, crop, 'Images'))
            filenames = [f.split('.')[0] for f in filenames]

            for curr_filename in filenames:
                
                if os.path.isfile(os.path.join(source_dir, team, crop, 'Images', curr_filename + '.png')):
                    img = np.array(Image.open(os.path.join(source_dir, team, crop, 'Images', curr_filename + '.png')).resize(TEAM_SIZES[team][0]))
                    ext = ".png"
                else:
                    img = np.array(Image.open(os.path.join(source_dir, team, crop, 'Images', curr_filename + '.jpg')).resize(TEAM_SIZES[team][0]))
                    ext = ".jpg"
                    
                mask = np.array(Image.open(os.path.join(source_dir, team, crop, 'Masks', curr_filename + '.png')).resize(TEAM_SIZES[team][0]))
               
                curr_dir = os.path.join(patch_dir, team, crop)
                if not os.path.exists(curr_dir):
                    os.makedirs(curr_dir)
                    os.makedirs(os.path.join(curr_dir, "Images"))
                    os.makedirs(os.path.join(curr_dir, "Masks"))

                #--------- salvo patches maschere e immagini ---------------------------------------------------
                for i in range(int(TEAM_SIZES[team][1]/scale)):
                    for j in range(int(TEAM_SIZES[team][1]/scale)):
                        pixels = mask[i*img_h*scale:(i+1)*img_h*scale,j*img_w*scale:(j+1)*img_w*scale,:]
                        
                        if tf.reduce_sum(tf.cast(np.all(pixels == [254, 124, 18], axis=-1),tf.float32) + tf.cast(np.all(pixels == [0, 0, 0], axis=-1),tf.float32)) <= img_h*scale*img_w*scale*PSI:
                            
                            #maschera
                            curr_patch_mask = Image.fromarray(pixels, 'RGB')
                            curr_patch_mask.save(os.path.join(curr_dir, 'Masks', curr_filename + "_" + str(i) + "_" + str(j) + "_" + str(scale) + ".png"))
                            
                            #immagine
                            curr_patch_img = Image.fromarray(img[i*img_h*scale:(i+1)*img_h*scale,j*img_w*scale:(j+1)*img_w*scale,:], 'RGB')
                            curr_patch_img.save(os.path.join(curr_dir, 'Images', curr_filename + "_" + str(i) + "_" + str(j) + "_" + str(scale) + ext))
        

In [ ]:
#--------------- creo i data generator -------------------------------------
from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

if apply_data_augmentation:
    #immagini
    img_data_gen = ImageDataGenerator(rotation_range=30,
                                      width_shift_range=0.1,
                                      height_shift_range=0.1,
                                      #brightness_range=[1,1.8],
                                      zoom_range=0.3,
                                      shear_range=10,
                                      horizontal_flip=True,
                                      vertical_flip=True,
                                      fill_mode='reflect')
    #maschere
    mask_data_gen = ImageDataGenerator(rotation_range=30,
                                      width_shift_range=0.1,
                                      height_shift_range=0.1,
                                      #brightness_range=[1,1.8],
                                      zoom_range=0.3,
                                      shear_range=10,
                                      horizontal_flip=True,
                                      vertical_flip=True,
                                      fill_mode='reflect')

else:
    img_data_gen = ImageDataGenerator(fill_mode='reflect')

    mask_data_gen = ImageDataGenerator(fill_mode='reflect')
    

In [ ]:
from PIL import Image

class CustomDataset(tf.keras.utils.Sequence):
    
    # costruttore: 
    def __init__(self, dataset_dir, team, crop, which_subset, subset_filenames, img_generator=None, mask_generator=None, 
               preprocessing_function=None, out_shape=[img_w,img_h]):
        
        self.which_subset = which_subset
        self.dataset_dir = dataset_dir
        self.subset_filenames = subset_filenames
        self.out_shape = out_shape
        self.img_generator = img_generator
        self.mask_generator = mask_generator
        self.preprocessing_function = preprocessing_function
        self.team = team
        self.crop = crop

    def __len__(self):
        return len(self.subset_filenames)
    
    def __getitem__(self, index):
        curr_filename = self.subset_filenames[index]
        
        if os.path.isfile(os.path.join(dataset_dir, team, crop, 'Images', curr_filename + '.png')):
            img = Image.open(os.path.join(dataset_dir, team, crop, 'Images', curr_filename + '.png'))
        else:
            img = Image.open(os.path.join(dataset_dir, team, crop, 'Images', curr_filename + '.jpg'))
            
        mask = Image.open(os.path.join(dataset_dir, team, crop, 'Masks', curr_filename + '.png'))
        
        img = img.resize(self.out_shape)
        mask = mask.resize(self.out_shape, resample=Image.NEAREST)

        img_arr = np.array(img)
        mask_arr = np.array(mask)
        # uso one-hot encoding con 4 classi
        new_mask_arr = np.zeros((mask_arr.shape[0], mask_arr.shape[1], 4))
        
        # trasfotmazione maschera e immagine
        if self.which_subset == 'training':
            if self.img_generator is not None and self.mask_generator is not None:
                img_t = self.img_generator.get_random_transform(img_arr.shape, seed=SEED)
                mask_t = self.mask_generator.get_random_transform(mask_arr.shape, seed=SEED)
                img_arr = self.img_generator.apply_transform(img_arr, img_t)
                mask_arr = np.uint8(self.mask_generator.apply_transform(mask_arr, mask_t))
        
        #tengo separati background e orange label
        new_mask_arr[np.where(np.all(mask_arr == [254, 124, 18], axis=-1))] = [0, 1, 0, 0] #background
        new_mask_arr[np.where(np.all(mask_arr == [255, 255, 255], axis=-1))] = [0, 0, 1, 0] #crop
        new_mask_arr[np.where(np.all(mask_arr == [216, 67, 82], axis=-1))] = [0, 0, 0, 1] #weed
        new_mask_arr[np.where(np.all(new_mask_arr == [0, 0, 0, 0], axis=-1))] = [1, 0, 0, 0] #backgroung
        
        if self.preprocessing_function is not None:
            img_arr = self.preprocessing_function(img_arr)

        return img_arr, np.float32(new_mask_arr)
    

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input 

#---------- funzione per random split (train/validation) -------------
def random_pick(filenames, percent):
    percent = np.clip(percent, 0, 1)
    subset = []
    n = int(percent*len(filenames))
    for i in range(0, n):
        index = np.random.randint(0, len(filenames))
        subset.append(filenames[index])
        del(filenames[index])        
    return subset

#-------- scegli dataset -----------------------------------------
dataset_dir = os.path.join(cwd,'Patches')
team = 'Pead'
crop = 'Haricot'
#-----------------------------------------------------------------
imm_source = os.path.join(dataset_dir, team, crop, 'Images')
subset_filenames = os.listdir(imm_source)
subset_filenames = [f.split('.')[0] for f in subset_filenames]

#subset_filenames = random_pick(subset_filenames, 0.3)
print("tot_imgs = " + str(len(subset_filenames)))

# seleziono immagini training
train_imm = random_pick(subset_filenames, split)
print("training_imgs = " + str(len(train_imm)))

# seleziono immagini training
valid_imm = []
for imm in subset_filenames:
    if imm not in train_imm:
        valid_imm.append(imm)        

print("validation_imgs = " + str(len(valid_imm)))



#-------- definisco datasets ----------------------------------------
dataset = CustomDataset(dataset_dir = dataset_dir,
                        team = team,
                        crop = crop,
                        which_subset = 'training',
                        subset_filenames = train_imm,
                        img_generator = img_data_gen,
                        mask_generator = mask_data_gen,
                        preprocessing_function = preprocess_input)

dataset_valid = CustomDataset(dataset_dir = dataset_dir,
                              team = team,
                              crop = crop,
                              which_subset = 'validation',
                              subset_filenames = valid_imm,
                              preprocessing_function=preprocess_input)

In [ ]:
#---------- train dataset -----------------------------------------------------------------
train_dataset = tf.data.Dataset.from_generator(lambda: dataset,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([img_h, img_w, 3], [img_h, img_w, 4]))
train_dataset = train_dataset.batch(BS)

train_dataset = train_dataset.repeat()

#------------ valid dataset ---------------------------------------------------------------
valid_dataset = tf.data.Dataset.from_generator(lambda: dataset_valid,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([img_h, img_w, 3], [img_h, img_w, 4]))
valid_dataset = valid_dataset.batch(BS)

valid_dataset = valid_dataset.repeat()

In [ ]:
from tensorflow.keras import layers
# modello --> U-net like: [256, 128, 64, 32] migliore per trial and error
def get_model(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)

    x = tf.keras.layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)

    previous_block_activation = x

    for filters in [64, 128, 256]:
        x = tf.keras.layers.Activation("relu")(x)
        x = tf.keras.layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)

        x = tf.keras.layers.Activation("relu")(x)
        x = tf.keras.layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)

        x = tf.keras.layers.MaxPooling2D(3, strides=2, padding="same")(x)

        residual = tf.keras.layers.Conv2D(filters, 1, strides=2, padding="same")(previous_block_activation)
        x = tf.keras.layers.add([x, residual])
        previous_block_activation = x

    for filters in [256, 128, 64, 32]:
        x = tf.keras.layers.Activation("relu")(x)
        x = tf.keras.layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)

        x = tf.keras.layers.Activation("relu")(x)
        x = tf.keras.layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)

        x = tf.keras.layers.UpSampling2D(2)(x)

        residual = tf.keras.layers.UpSampling2D(2)(previous_block_activation)
        residual = tf.keras.layers.Conv2D(filters, 1, padding="same")(residual)
        x = tf.keras.layers.add([x, residual])
        previous_block_activation = x

    outputs = tf.keras.layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    model = tf.keras.Model(inputs, outputs)
    return model

tf.keras.backend.clear_session()

In [ ]:
#-------------- creo il modello ----------------------------------------------------------------
teams = ["Bipbip", "Pead", "Roseau", "Weedelec"]
crops = ["Haricot", "Mais"]

model = get_model((img_h, img_w, 3), N_classes)

#carico i pesi
load_weights = True
if load_weights:
    model_name = teams[0] + crops[0] #<-- scegli quale TEAM/CROP caricare
    model_dir = os.path.join(cwd, 'weights_4D', model_name)
    model.load_weights(os.path.join(model_dir, model_name))




In [ ]:
#------------ IoU(intersect over union): metodo per valutare la performance --------------------
# non peso IoU per poter visualizzare valore verosimile
def meanIoU(y_true, y_pred):
    per_class_iou = []
    y_pred = tf.cast(y_pred, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    # non includo ne orange labels ne background
    for i in range(2,N_classes):       
        intersection = tf.reduce_sum(y_true[:,:,i] * y_pred[:,:,i])
        union = tf.reduce_sum(y_true[:,:,i]) + tf.reduce_sum(y_pred[:,:,i]) - intersection
        iou = ((intersection + 1e-7) / (union + 1e-7))
        per_class_iou.append(iou)

    return tf.reduce_mean(per_class_iou)

In [ ]:
#-------------- custom loss --> weighted_pixelwise_crossentropy ------------------------------
def weighted_pixelwise_crossentropy(class_weights):
    _EPSILON = 1e-7
    def loss(y_true, y_pred):
        epsilon = tf.convert_to_tensor(_EPSILON, y_pred.dtype.base_dtype)
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        return -tf.reduce_sum(tf.multiply(y_true * tf.math.log(y_pred), class_weights))
    
    return loss

In [ ]:
#-------------- custom loss --> diagonal_pixelwise_crossentropy ------------------------------
def diagonal_pixelwise_crossentropy(class_weights):
    _EPSILON = 1e-7
    
    def loss(y_true, y_pred):
        epsilon = tf.convert_to_tensor(_EPSILON, y_pred.dtype.base_dtype)        
        per_class_loss = []
        for i in range(0,N_classes):            
            slice_pred = tf.clip_by_value(y_pred[:,:,i], epsilon, 1. - epsilon)
            slice_true = y_true[:,:,i]
            tmp = -tf.reduce_sum(tf.multiply(slice_true * tf.math.log(slice_pred), class_weights[i]))
            per_class_loss.append(tmp)            
        return tf.reduce_mean(per_class_loss)
    
    return loss

In [ ]:
#-------------- custom loss --> IoU_crossentropy ------------------------------
def IoU_crossentropy(class_weights):
    
    _EPSILON = 1e-9
    
    def loss(y_true, y_pred):
        epsilon = tf.convert_to_tensor(_EPSILON, y_pred.dtype.base_dtype)
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        intersection = -tf.reduce_sum(y_true * tf.math.log(y_pred))
        union = tf.clip_by_value(tf.reduce_sum(y_true) + tf.reduce_sum(y_pred), epsilon, float("inf"))
        return tf.reduce_mean(tf.multiply(intersection / union, class_weights))
    
    return loss

In [ ]:
#-------------- custom loss --> UoI_crossentropy ------------------------------
def UoI_loss(class_weights):
    
    _EPSILON = 1e-9
    
    def loss(y_true, y_pred):
        epsilon = tf.convert_to_tensor(_EPSILON, y_pred.dtype.base_dtype)
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        intersection = tf.clip_by_value(tf.reduce_sum(y_true * y_pred), epsilon, float("inf"))
        union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
        return tf.reduce_mean(tf.multiply(union / intersection, class_weights))
    
    return loss

In [ ]:
#-------- classe per cambiare lr ----------------------
class CLR(tf.keras.callbacks.Callback):
    def __init__(self, schedule):
        super(CLR, self).__init__()
        self.schedule = schedule
    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('non hai settato lr')
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        scheduled_lr = self.schedule(epoch, lr)
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)

In [ ]:
#---------------- definisco callbacks ------------------------------------
from tensorflow.keras.callbacks import EarlyStopping
    
callbacks = []

early_stop = True
if early_stop:
    es_callback = EarlyStopping(monitor='val_loss', patience=20)
    callbacks.append(es_callback)

#--------------------------- lookup table per lr (standard)-------------
LUT_STD = []
#------------------- funzione per passare lr ---------------------------
def get_lr_std(epoch, lr):
    if epoch < LUT_STD[0][0]:
        return LUT_STD[0][1]
    elif epoch > LUT_STD[len(LUT_STD)-1][0]:
        return LUT_STD[len(LUT_STD)-1][1]
    for i in range(len(LUT_STD)):
        if epoch == LUT_STD[i][0]:
            print("\nnuovo lr: "+str(LUT_STD[i][1]))
            return LUT_STD[i][1]
    return lr

callbacks.append(CLR(get_lr_std))


In [ ]:
#---------- one hot encoders ------------------------------------------------
# [1, 0, 0, 0] = background
# [0, 1, 0, 0]  = background
# [0, 0, 1, 0]  = crop
# [0, 0, 0, 1]  = weed

class_weights = [1, 10, 180, 90]
#----------------------------------------------------------------------------
# allenate con loss nell'ordine: 1) --> 2) --> 3)
loss = tf.keras.losses.CategoricalCrossentropy()  #1)
#loss = IoU_crossentropy(class_weights)  #2)
#loss = UoI_loss(class_weights)  #3)

#loss= weighted_pixelwise_crossentropy(class_weights)  # per test)
#loss = diagonal_pixelwise_crossentropy(class_weights)  #per test)

metrics = ['accuracy', meanIoU]

model.compile(optimizer='adam',
              loss=loss,
              metrics=metrics)

In [ ]:
#----------- fitto modello ----------------------------------------
EP = 20

# <-- qua scegli i learning rate per epochs
LUT_STD = [(0, 1e-3),
           (10, 1e-4),
           (20, 1e-6)]

model.fit(x=train_dataset,
          epochs=EP,
          steps_per_epoch=len(dataset),
          validation_data=valid_dataset,
          validation_steps=len(dataset_valid), 
          callbacks=callbacks)

In [ ]:
#------------- salvo pesi --------------------------------------------------
save_weights = True

if save_weights:
    model_name = team + crop
    model_dir = os.path.join(cwd, 'weights_4D')
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)    
    model.save_weights(os.path.join(model_dir, model_name))

In [ ]:
#----- funzioni per riscalare immagine ------------------------------------
import scipy
#---------- riscalo per target size (no interpolazione) -------------------
def MAP(arr, X):
    (w,h) = arr.shape
    new_arr = np.zeros((X*w,X*h), dtype=arr.dtype)
    for j in range(w):
        for i in range(h):
            new_arr[j*X:X*(j+1), i*X:X*(i+1)] = arr[j,i]
            
    return new_arr

#------- zoom a target size (con bilinear interpolazione) ----------------
def zoom_target(out_sigmoid, scale):
    big_sigmoid = np.zeros((scale*out_sigmoid.shape[0], scale*out_sigmoid.shape[1], 3))
    for i in range(out_sigmoid.shape[2]):
        big_sigmoid[:,:,i] = scipy.ndimage.zoom(out_sigmoid[:,:,i], scale, order=3)
    return tf.argmax(big_sigmoid, -1)

#------- resize a target size (con bilinear interpolazione) --------------
def resize_target(out_sigmoid, target_shape):
    big_sigmoid = np.zeros((target_shape[0], target_shape[1], target_shape[2]), dtype=np.float32)
    for i in range(out_sigmoid.shape[2]):
        big_sigmoid[:,:,i] = np.array(Image.fromarray(out_sigmoid[:,:,i]).resize((target_shape[1], target_shape[0])))
    return big_sigmoid

In [ ]:
#--------- preparo per plottare ---------------------------------------------
import time
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
item = 0

In [ ]:
# PLOTTO: IMMAGINE vs TRUE MASK vs PREDICTED MASK vs ZOOMED MASK

fig, ax = plt.subplots(1, 4, figsize=(32, 32))
ax[0].set_title('reconstructed image')
ax[1].set_title('true mask')
ax[2].set_title('predicted mask')
ax[3].set_title('zoomed prediction')
fig.show()


dataset_dir = os.path.join(cwd,'Development_Dataset', 'Training')
teams = ["Bipbip", "Pead", "Roseau", "Weedelec"]
crops = ["Haricot", "Mais"]

TEAM_SIZES = {"Bipbip": [(2048, 1536), 8],
              "Pead": [(3328, 2496), 13],
              "Roseau": [(1280, 960), 5],
              "Weedelec": [(5120, 3840), 20]}

TARGET_SIZES = {"Bipbip": (1536, 2048, 4),
                "Pead": (2464, 3280, 4),
                "Roseau": (820, 1227, 4),
                "Weedelec": (3456, 5184, 4)}


#------------- scegli TEAM/CROP e se portare a target -----------
toTarget = True
standard_dim = False
team = teams[1] # --> TEAM
crop = crops[0] # --> CROP

scale = LUT_scale[team + crop]

#----------------------------------------------------------------

filenames = os.listdir(os.path.join(dataset_dir, team, crop, 'Images'))
filenames = [f.split('.')[0] for f in filenames]

curr_filename = filenames[item]
item+=1

#--------------- immagine a patch -------------------------------------------------------------------------
    
if os.path.isfile(os.path.join(dataset_dir, team, crop, 'Images', curr_filename + '.png')):
    image = preprocess_input(np.array(Image.open(os.path.join(dataset_dir, team, crop, 'Images', curr_filename + '.png')).resize(TEAM_SIZES[team][0])))
else:
    image = preprocess_input(np.array(Image.open(os.path.join(dataset_dir, team, crop, 'Images', curr_filename + '.jpg')).resize(TEAM_SIZES[team][0])))

mask = np.array(Image.open(os.path.join(dataset_dir, team, crop, 'Masks', curr_filename + '.png')).resize(TEAM_SIZES[team][0]))

out_sigmoid = np.zeros((1, img_h*int(TEAM_SIZES[team][1]/scale), img_w*int(TEAM_SIZES[team][1]/scale), 4))

# predico ogni patch e la posiziono nella parte rispettiva di out_sigmoid
for i in range(int(TEAM_SIZES[team][1]/scale)):
    for j in range(int(TEAM_SIZES[team][1]/scale)):
        patch = resize_target(image[i*scale*img_h:(i+1)*scale*img_h,j*scale*img_w:(j+1)*scale*img_w,:], (img_h, img_w, 3))
        out_sigmoid[0, i*img_h:(i+1)*img_h,j*img_w:(j+1)*img_w,:] = model.predict(x = tf.expand_dims(patch,0))[0]

#-----------------------------------------------------------------------------------------------------------

# lascio separato [1, 0, 0, 0] da [0, 1, 0, 0] per visualizzare differenza
prediction = tf.argmax(out_sigmoid, -1)[0, ...].numpy()

if toTarget:
    zoom = tf.argmax(resize_target(out_sigmoid[0, ...], TARGET_SIZES[team]), -1).numpy()
    # unisco orange a background
    zoom[np.where(zoom == 1)] = 0 

# colori per visualizzare
LUT = [[0, 0, 0],
       [254, 124, 18],
       [255, 255, 255],
       [255, 0, 0]]
    
prediction_img = np.zeros([prediction.shape[0], prediction.shape[1], 3])
zoomed_img = np.zeros([zoom.shape[0], zoom.shape[1], 3])

#-------------- predizione --------------------------------------------------
for i in range(0, N_classes):
    prediction_img[np.where(prediction == i)] = LUT[i]
#------------ predizione zummata ---------------------------------------------
if toTarget:
    for i in range(0, N_classes):
        zoomed_img[np.where(zoom == i)] = LUT[i]
        
ax[0].imshow(np.uint8(image))
ax[1].imshow(np.uint8(mask))
ax[2].imshow(np.uint8(prediction_img))
if toTarget:
    ax[3].imshow(np.uint8(zoomed_img))

fig.canvas.draw()
time.sleep(1)

In [ ]:
#------------- faccio predizioni e preparo submission --------------------------
import json
from tensorflow.keras.applications.vgg16 import preprocess_input

#---------------- encode function ------------------------------------------
def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

#---------------- prepare submission ---------------------------------------
def prepare_img(dataset_dir, team, crop, curr_filename, TEAM_SIZES, TARGET_SIZES, scale, preprocess_input):
    if os.path.isfile(os.path.join(dataset_dir, team, crop, 'Images', curr_filename + '.png')):
        image = preprocess_input(np.array(Image.open(os.path.join(dataset_dir, team, crop, 'Images', curr_filename + '.png')).resize(TEAM_SIZES[team][0])))
    else:
        image = preprocess_input(np.array(Image.open(os.path.join(dataset_dir, team, crop, 'Images', curr_filename + '.jpg')).resize(TEAM_SIZES[team][0])))

    out_sigmoid = np.zeros((1, img_h*int(TEAM_SIZES[team][1]/scale), img_w*int(TEAM_SIZES[team][1]/scale), 4))

    for i in range(int(TEAM_SIZES[team][1]/scale)):
        for j in range(int(TEAM_SIZES[team][1]/scale)):
            patch = resize_target(image[i*scale*img_h:(i+1)*scale*img_h,j*scale*img_w:(j+1)*scale*img_w,:], (img_h, img_w, 3))
            out_sigmoid[0, i*img_h:(i+1)*img_h,j*img_w:(j+1)*img_w,:] = model.predict(x = tf.expand_dims(patch,0))[0]

    return out_sigmoid
#----------------------------------------------------------------------------       
    
#-------- dati --------------------------------------------------
teams = ["Bipbip", "Pead", "Roseau", "Weedelec"]
crops = ["Haricot", "Mais"]

TARGET_SIZES = {"Bipbip": (1536, 2048, 4),
                "Pead": (2464, 3280, 4),
                "Roseau": (820, 1227, 4),
                "Weedelec": (3456, 5184, 4)}

dataset_dir = os.path.join(cwd,'Development_Dataset', 'Test_Dev')
prediction_dir = os.path.join(cwd, 'predictions')


#------------ predizioni per Test set ----------------------------
submission_dict = {}
for team in teams:
    print(team)
    for crop in crops:
        print(crop)
        # aggiorna la scala a seconda di TEAM e CROP
        scale = LUT_scale[team + crop]
        imm_source = os.path.join(dataset_dir, team, crop, 'Images')
        filenames = os.listdir(imm_source)
        filenames = [f.split('.')[0] for f in filenames]
        
        for curr_filename in filenames:
            print(curr_filename)
            #preprocessing
            out_sigmoid = prepare_img(dataset_dir, team, crop, curr_filename, TEAM_SIZES, TARGET_SIZES, scale, preprocess_input)  
            
            predicted_class = tf.argmax(resize_target(out_sigmoid[0, ...], TARGET_SIZES[team]), -1).numpy()
            #riporto le calssi standard per usare rle_encode senza modificarlo (potevo cambiare indici anche li)
            predicted_class[np.where(predicted_class == 1)] = 0
            predicted_class[np.where(predicted_class == 2)] = 1
            predicted_class[np.where(predicted_class == 3)] = 2
            
            # costruisco il dict
            submission_dict[curr_filename] = {}
            submission_dict[curr_filename]['shape'] = [TARGET_SIZES[team][0], TARGET_SIZES[team][1]]
            submission_dict[curr_filename]['team'] = team
            submission_dict[curr_filename]['crop'] = crop
            submission_dict[curr_filename]['segmentation'] = {}
            
            # distinguo tra crop e weed
            rle_encoded_crop = rle_encode(predicted_class == 1) # crop
            rle_encoded_weed = rle_encode(predicted_class == 2) # weed
            
            # trascrivo rle_encoded_crop e rle_encoded_weed
            submission_dict[curr_filename]['segmentation']['crop'] = rle_encoded_crop
            submission_dict[curr_filename]['segmentation']['weed'] = rle_encoded_weed

# salvo predictions in json
if not os.path.exists(prediction_dir):
        os.makedirs(prediction_dir)
with open(os.path.join(prediction_dir,'submission.json'), 'a+') as f:
    json.dump(submission_dict, f)
print("predictions saved")